In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import pandas as pd
import plotly.express as px

# Load data
data = pd.read_csv(
    '/Users/saikatbasu/Documents/PersonalInformation_Oct2022/IBMDataScience-Coursera/AppliedDataScience-10of12/Module3-InteractiveVisualAnalytics/spacex_launch_dash.csv'
)

#data.head()
max_payload = data['Payload Mass (kg)'].max()
min_payload = data['Payload Mass (kg)'].min()
sites = data['Launch Site'].unique()

for site in sites:
    print (site)
    

CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [2]:
app = dash.Dash(__name__)

app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id ='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                            ],
                                value='ALL',
                                placeholder="ALL Sites",
                                searchable=True
                                ),
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                
                                dcc.Graph(id='success-pie-chart'),
                                
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000,
                                marks={0: '0', 10000: '10000'},
                                value=[min_payload, max_payload]),

                                html.Br(),  
                                
                                dcc.Graph(id='success-scatter-chart'),
                                
                                
                               ])

In [3]:
# Callback for dropdown list of Launch Sites

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = data
    if entered_site == 'ALL':
        fig = px.pie(data, values='class', names='Launch Site') 
        title='Landing Success Rate'
        return fig
    else:
        df = data[data['Launch Site'] == entered_site]
        #print(len(data['class'].value_counts()))
        val = df['class'].value_counts()
        nm = ['Failure', 'Succcess']
        fig = px.pie(df, names=nm, values=val, title = site)
        return fig

In [4]:
# Callback for slide-ranger for choosing Payload mass

@app.callback(Output(component_id='success-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')]
             )

def get_scatter_plot(entered_site,slider_range):
    #filtered_df = data
    if entered_site == 'ALL':
        low,high = slider_range
        filtered_df = data[(data['Payload Mass (kg)'] > low) & (data['Payload Mass (kg)'] <=high)]
        fig = px.scatter(x=filtered_df['Payload Mass (kg)'], y = filtered_df['class'], color=filtered_df['Booster Version']) 
        title='Payload mass Vs. Outcome'
        return fig
    else:
        low,high = slider_range
        df = data[data['Launch Site'] == entered_site]
        df1 = df[(df['Payload Mass (kg)'] > low) & (data['Payload Mass (kg)'] < high)]
        fig = px.scatter(x=df1['Payload Mass (kg)'], y = df1['class'], color=filtered_df['Booster Version']) 
        title='Payload mass Vs. Outcome'
        return fig

In [5]:
# Run
if __name__ == '__main__':
    app.run(debug=False)